First I imported the libraries

In [59]:
#import libraries that i will use
import pandas as pd
import numpy as np

I loaded files

In [60]:
#here is the file load:
file_path = "daten.xlsx"
xls = pd.ExcelFile(file_path)


### I saw the sheets 

In [61]:
# Print sheet names to inspect
print("Available Sheets:", xls.sheet_names)

Available Sheets: ['bad_data', 'data']


In [62]:
# I loaded the data sheets skipping first some rows
df = pd.read_excel(xls, sheet_name='data', skiprows=3)

I printed dataframe of existed data

In [63]:
print(df)

                    Datum  Input Strom [A]  Anlagetemperatur [°C]  \
0     2024-11-10 15:11:00              NaN                  63.24   
1     2024-11-10 15:21:00              NaN                  63.24   
2     2024-11-10 15:31:00              NaN                  63.24   
3     2024-11-10 15:41:00              NaN                  63.24   
4     2024-11-10 15:51:00              NaN                  63.24   
...                   ...              ...                    ...   
17766 2024-12-13 09:26:00              9.5                  65.72   
17767 2024-12-13 09:31:00              9.5                  65.10   
17768 2024-12-13 09:35:00              9.5                  65.10   
17769 2024-12-13 09:36:00              9.1                  65.10   
17770 2024-12-13 09:41:00              9.1                  65.10   

       Dichte  [g/m³]  Rel. Feuchte [%]  
0            0.000000               NaN  
1            0.000000               NaN  
2            0.000000               NaN  
3  

In [64]:
df = df.dropna(subset=['Datum'])

In [65]:
df.set_index('Datum', inplace=True)

Created timestamps for every 10 minutes.

In [66]:
start = df.index.min().floor('10min') + pd.Timedelta(minutes=1)
end = df.index.max().ceil('10min')
target_times = pd.date_range(start=start, end=end, freq='10min')

printed targeted times

In [67]:
print(target_times)

DatetimeIndex(['2024-11-10 15:11:00', '2024-11-10 15:21:00',
               '2024-11-10 15:31:00', '2024-11-10 15:41:00',
               '2024-11-10 15:51:00', '2024-11-10 16:01:00',
               '2024-11-10 16:11:00', '2024-11-10 16:21:00',
               '2024-11-10 16:31:00', '2024-11-10 16:41:00',
               ...
               '2024-12-13 08:11:00', '2024-12-13 08:21:00',
               '2024-12-13 08:31:00', '2024-12-13 08:41:00',
               '2024-12-13 08:51:00', '2024-12-13 09:01:00',
               '2024-12-13 09:11:00', '2024-12-13 09:21:00',
               '2024-12-13 09:31:00', '2024-12-13 09:41:00'],
              dtype='datetime64[ns]', length=4720, freq='10min')


Check the closest interval

In [68]:
def get_closest_data(ts):
    interval_start = ts - pd.Timedelta(minutes=1)
    interval_end = ts + pd.Timedelta(minutes=9)
    mask = (df.index >= interval_start) & (df.index < interval_end)
    if mask.any():
        return df.loc[mask].iloc[[0]]  # Pick any one (first) if multiple exist
    else:
        return None

Convert it to homogenised data

In [69]:
homogenized_rows = []
last_valid_row = None

for ts in target_times:
    row = get_closest_data(ts)
    if row is not None:
        homogenized_rows.append(row.set_index(pd.DatetimeIndex([ts])))
        last_valid_row = row
    elif last_valid_row is not None:
        homogenized_rows.append(last_valid_row.set_index(pd.DatetimeIndex([ts])))

In [70]:
# Combine all rows into a single DataFrame
homogenized_df = pd.concat(homogenized_rows)

In [71]:
homogenized_df.reset_index(inplace=True)
homogenized_df.rename(columns={'index': 'Datum'}, inplace=True)

In [72]:
print("Final Homogenized Data:")
display(homogenized_df.head(20))

Final Homogenized Data:


,Datum,Input Strom [A],Anlagetemperatur [°C],Dichte [g/m³],Rel. Feuchte [%]
0,2024-11-10 15:11:00,NaN,63.24,0.0,NaN
1,2024-11-10 15:21:00,NaN,63.24,0.0,NaN
2,2024-11-10 15:31:00,NaN,63.24,0.0,NaN
3,2024-11-10 15:41:00,NaN,63.24,0.0,NaN
4,2024-11-10 15:51:00,NaN,63.24,0.0,NaN
5,2024-11-10 16:01:00,NaN,63.24,0.0,NaN
6,2024-11-10 16:11:00,NaN,63.24,0.0,NaN
7,2024-11-10 16:21:00,NaN,63.24,0.0,NaN
8,2024-11-10 16:31:00,NaN,63.24,0.0,NaN
9,2024-11-10 16:41:00,NaN,63.24,0.0,NaN


Output dataframe to excel file

In [73]:
# Save homogenized DataFrame to Excel
output_path = "homogene_daten.xlsx"
homogenized_df.to_excel(output_path, index=False)
